In [101]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [102]:
df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv")
df_hedged_returns.set_index('date', inplace=True)

df_return_with_div = pd.read_csv("data/preprocess/merged_returns_with_dividends.csv")
df_return_with_div.set_index('date', inplace=True)

df = df_hedged_returns.rename(columns={
    "australia_hedged_return_usd":   "AUS",
    "france_hedged_return_usd":      "FRA",
    "germany_hedged_return_usd":     "GER",
    "japan_hedged_return_usd":       "JPN",
    "switzerland_hedged_return_usd": "SWI",
    "uk_hedged_return_usd":          "UK"
})

df['US'] = df_return_with_div["us_ret_with_div_usd"]

# Convert the 'date' column to datetime format and shift to the first day of the next month
df_tbills = pd.read_csv('data/preprocess/tbills.csv')
df_tbills.set_index('date', inplace=True)

# To match the hedged return we drop the first row
tbill = df_tbills.rename(columns={"rf": "TBill1Mo"})["TBill1Mo"].iloc[1:]


div_df = pd.read_csv("data/q3/div_return.csv")
div_df.set_index('date', inplace=True)
div = div_df["DIV_return"]   


one_plus = 1.0 + df

In [103]:
# Helper to compute annualized mean, vol, and Sharpe
def annualized_stats(returns, rf = None, period = 12, zero_cost = False):
    """
    Given two pandas Series of monthly returns (decimal):
      - returns: strategy returns
      - rf:      risk-free returns
    Returns (annualized_mean, annualized_vol, annualized_sharpe).
    """
    mean_annual_return = period * returns.mean()
    std_annual_return = np.sqrt(period) * returns.std()
    if zero_cost:
        return mean_annual_return, std_annual_return, (mean_annual_return/std_annual_return)
    rf_annual = period * rf.mean()
    excess_ret = mean_annual_return - rf_annual
    sharpe_ratio = excess_ret / std_annual_return
    return mean_annual_return, std_annual_return, sharpe_ratio

### 4 Equity Index Momentum Strategy (MOM)

In [104]:
# a)

# Compute 11-month lookback cumulative return, then shift by 1 to impose 1-month lag:
cumprod_11 = one_plus.rolling(window=12).apply(np.prod).shift(1) - 1

# Rank each row (month) ascending
ranks_11 = cumprod_11.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N_mom = 7            
mean_rank_mom = (N_mom + 1) / 2.0   

# Compute MOM weights
raw_w_mom = (ranks_11 - mean_rank_mom)
Z =   1.0 / 6.0
w_mom = raw_w_mom * Z

# Strategy return at month 
mom_ret = (w_mom * df).sum(axis=1, skipna=False)

# Decompose the “long leg” (w>0) and “short leg” (w<0) returns:
long_w_mom  = w_mom.clip(lower=0)
short_w_mom = w_mom.clip(upper=0)

mom_long_ret  = (long_w_mom  * df).sum(axis=1, skipna=False)
mom_short_ret = (short_w_mom * df).sum(axis=1, skipna=False)

# Align with T-Bill series, drop NaNs (the first 12 months) for clean statistics:
mom_df = pd.DataFrame({
    "MOM":   mom_ret,
    "LONG":  mom_long_ret,
    "SHORT": mom_short_ret,   
    "TBill": tbill
}).dropna()


In [105]:
# b)

mean_mom, sigma_mom, sharpe_mom = annualized_stats(mom_df["MOM"], zero_cost=True)
mean_mL,  sigma_mL,  sharpe_mL  = annualized_stats(mom_df["LONG"], rf = mom_df["TBill"])
mean_mS,  sigma_mS,  sharpe_mS  = annualized_stats(mom_df["SHORT"], rf = mom_df["TBill"])

# One-sample t-test: H0: mean(MOM) = 0
t_stat_mom, p_val_mom = stats.ttest_1samp(mom_df["MOM"], popmean=0.0)

print("=== MOMENTUM (MOM) Strategy Statistics ===")
print(f"Overall MOM       : Annualized Return = {mean_mom:.2%}, "
      f"Volatility = {sigma_mom:.2%}, Sharpe = {sharpe_mom:.2f}")
print(f"MOM Long Leg      : Annualized Return = {mean_mL:.2%}, "
      f"Volatility = {sigma_mL:.2%}, Sharpe = {sharpe_mL:.2f}")
print(f"MOM Short Leg     : Annualized Return = {mean_mS:.2%}, "
      f"Volatility = {sigma_mS:.2%}, Sharpe = {sharpe_mS:.2f}")
print(f"MOM mean-return t-stat = {t_stat_mom:.3f}, p-value = {p_val_mom:.3f}\n")

alpha = 0.05  # typical significance level
if p_val_mom < alpha:
    print("The strategy's mean return is significantly different from zero.")
else:
    print("The strategy's mean return is not significantly different from zero.")


mom_df['MOM'].to_csv('data/q4/mom_return.csv')

=== MOMENTUM (MOM) Strategy Statistics ===
Overall MOM       : Annualized Return = 27.58%, Volatility = 8.14%, Sharpe = 3.39
MOM Long Leg      : Annualized Return = 26.26%, Volatility = 13.98%, Sharpe = 1.77
MOM Short Leg     : Annualized Return = 1.32%, Volatility = 13.67%, Sharpe = -0.01
MOM mean-return t-stat = 15.751, p-value = 0.000

The strategy's mean return is significantly different from zero.


In [106]:
# c)

# Regress MOM on DIV (Newey-West errors, lag=1)
reg_mom_div = pd.DataFrame({
    "MOM": mom_ret.iloc[48:],
    "DIV": div
}).dropna()

X_m = sm.add_constant(reg_mom_div["DIV"])
y_m = reg_mom_div["MOM"]
mom_on_div = sm.OLS(y_m, X_m).fit()

print(">>> Regression of MOM on DIV:")
print(mom_on_div.summary())
print("\n")

>>> Regression of MOM on DIV:
                            OLS Regression Results                            
Dep. Variable:                    MOM   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7791
Date:              ven., 13 juin 2025   Prob (F-statistic):              0.378
Time:                        12:13:27   Log-Likelihood:                 500.05
No. Observations:                 211   AIC:                            -996.1
Df Residuals:                     209   BIC:                            -989.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0193 

### 5 Equity Index Long Term Reversal strategy (REV)

In [107]:
# a)

# Compute rolling 60-month product and rolling 12-month product:
rolling48 = one_plus.rolling(window=60).apply(lambda x: np.prod(x[:-11])).shift(1) - 1

# Rank each month ascending 
ranks_48 = rolling48.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N_rev = 7            
mean_rank_rev = (N_rev + 1) / 2.0    

# Compute MOM weights
raw_w_rev = (mean_rank_rev - ranks_48)
Z = (1.0 / 6.0)
w_rev = Z * raw_w_rev

# Strategy return at t: sum_i [ w_rev[i,t] * df[i,t] ]
rev_ret = (w_rev * df).sum(axis=1, skipna=False)

# Decompose “long leg” and “short leg” for REV:
long_w_rev  = w_rev.clip(lower=0)
short_w_rev = w_rev.clip(upper=0)
rev_long_ret  = (long_w_rev  * df).sum(axis=1, skipna=False)
rev_short_ret = (short_w_rev * df).sum(axis=1, skipna=False)

# Align with T-Bill, drop NaNs (the first 60 months):
rev_df = pd.DataFrame({
    "REV":   rev_ret,
    "LONG":  rev_long_ret,
    "SHORT": rev_short_ret,
    "TBill": tbill
}).dropna()

In [108]:
# b) 

mean_rev, sigma_rev, sharpe_rev = annualized_stats(rev_df["REV"], zero_cost=True)
mean_rL,  sigma_rL,  sharpe_rL  = annualized_stats(rev_df["LONG"], rf = rev_df["TBill"])
mean_rS,  sigma_rS,  sharpe_rS  = annualized_stats(rev_df["SHORT"], rf = rev_df["TBill"])

# One-sample t-test for REV mean
t_stat_rev, p_val_rev = stats.ttest_1samp(rev_df["REV"], popmean=0.0)

print("=== REVERSAL (REV) Strategy Statistics ===")
print(f"Overall REV       : Annualized Return = {mean_rev:.2%}, "
      f"Volatility = {sigma_rev:.2%}, Sharpe = {sharpe_rev:.2f}")
print(f"REV Long Leg      : Annualized Return = {mean_rL:.2%}, "
      f"Volatility = {sigma_rL:.2%}, Sharpe = {sharpe_rL:.2f}")
print(f"REV Short Leg     : Annualized Return = {mean_rS:.2%}, "
      f"Volatility = {sigma_rS:.2%}, Sharpe = {sharpe_rS:.2f}")
print(f"REV mean-return t-stat = {t_stat_rev:.3f}, p-value = {p_val_rev:.3f}\n")

alpha = 0.05  # typical significance level

if p_val_rev < alpha:
    print("The strategy's mean return is significantly different from zero.")
else:
    print("The strategy's mean return is not significantly different from zero.")

rev_df['REV'].to_csv('data/q5/rev_return.csv')

=== REVERSAL (REV) Strategy Statistics ===
Overall REV       : Annualized Return = -20.55%, Volatility = 7.45%, Sharpe = -2.76
REV Long Leg      : Annualized Return = -1.27%, Volatility = 14.37%, Sharpe = -0.17
REV Short Leg     : Annualized Return = -19.28%, Volatility = 14.04%, Sharpe = -1.46
REV mean-return t-stat = -11.567, p-value = 0.000

The strategy's mean return is significantly different from zero.


In [109]:
# c)

# Regress REV on DIV (Newey-West lag=1)
reg_rev_div = pd.DataFrame({
    "REV": rev_ret,
    "DIV": div
}).dropna()

X_r = sm.add_constant(reg_rev_div["DIV"])
y_r = reg_rev_div["REV"]
rev_on_div = sm.OLS(y_r, X_r).fit()

print(">>> Regression of REV on DIV:")
print(rev_on_div.summary())

>>> Regression of REV on DIV:
                            OLS Regression Results                            
Dep. Variable:                    REV   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.4703
Date:              ven., 13 juin 2025   Prob (F-statistic):              0.494
Time:                        12:13:27   Log-Likelihood:                 511.43
No. Observations:                 211   AIC:                            -1019.
Df Residuals:                     209   BIC:                            -1012.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0174 